# Establish Error Bars on Stopping Force
[Sand et al.](https://www.nature.com/articles/s41524-019-0180-5#Fig4) established that projectiles traveling along a channel oscillate around the center of the channel. Here, we establish the error bar in the stopping power for a channel due to this variation.

In [1]:
from scipy.integrate import dblquad
import pickle as pkl
import numpy as np

## Load in Trajectory Computer
We need the trajectory computer to quickly evaluate the acerage stopping power along a trajectory using adaptive integration.

In [2]:
with open('traj_computer.pkl', 'rb') as fp:
    traj_computer = pkl.load(fp)

## Set Up Calculator
Our goal is to compute the standard deviation of stopping power over samples of different deviations from the original starting point.
We will assume that the projectile can be anywhere within a certain distance, $r$, of the center of the channel

We do that by computing two integrals: 

1. $\mu = \frac{1}{\pi r^2 d} \int_0^r \int_0^{2\pi} \int_0^d F(r\cos\theta, r\sin\theta, z) r dz d\theta dr$
2. $\sigma^2 = \frac{1}{\pi r^2 d} \int_0^r \int_0^{2\pi} \int_0^d (F(r\cos\theta, r\sin\theta, z) - \mu)^2 r dz d\theta dr$

where $F(x, y, z)$ is the stopping force at a certain Cartesian coordinate, $d$ is the lattice period for the particular trajectory,
and the cylindrical coordinate system defined by $(r, \theta, z)$ is oriented with the z-axis along the projectile's direction of travel.

We already have an efficent integrator for inner integral (force along the direction of travel) and can use a similar adaptive integration scheme
to integrate over the other two variables ($r$, $\theta$)


In [3]:
def compute_stopping_distribution(
    start_point,
    lattice_vector, 
    velocity,
    radius: float,
    **kwargs
):
    """Compute the mean and standard deviation of stopping power for a certain channel,
    assuming that a projectile can move away from the channel center
    
    Keyword arguments are passed to `traj_computer.compute_stopping_power`
    
    Args:
        start_point: Starting position for the project
        lattice_vector: Direction of travel
        velocity: Projectile velocity
        radius: How far from the center the projectile can vary in fractional coordinates
    """

    # Determine orthogonal vectors for "x" and "y"
    #  We are using a cubic cell in this study, so I know the axes are aligned with the coordinate system
    nonzero_ind = [x != 0 for x in lattice_vector].index(True)
    to_swap = (nonzero_ind + 1) % 3
    ortho_vector_x = lattice_vector.copy()
    ortho_vector_x[nonzero_ind], ortho_vector_x[to_swap] = ortho_vector_x[to_swap], -ortho_vector_x[nonzero_ind]
    ortho_vector_y = np.cross(lattice_vector, ortho_vector_x)
    assert np.dot(lattice_vector, ortho_vector_x) == 0
    assert np.dot(lattice_vector, ortho_vector_y) == 0
    
    # Make them unit vectors
    ortho_vector_x = np.divide(ortho_vector_x, np.linalg.norm(ortho_vector_x))
    ortho_vector_y = np.divide(ortho_vector_y, np.linalg.norm(ortho_vector_y))
    
    # Make a function which computes the stopping power along a perturbed channel
    def _perturbed(r, theta):
        new_start_point = np.array(start_point)
        new_start_point += r * (ortho_vector_x * np.cos(theta) + ortho_vector_y * np.sin(theta))
        result = traj_computer.compute_stopping_power(
            new_start_point, lattice_vector, velocity, **kwargs
        )[0]  # Only the mean
        return result
    
    # Compute the mean and then the standard deviation
    mean, _ = dblquad(lambda r, t: r * _perturbed(r, t), 0, 2 * np.pi, 0, radius, epsabs=0.001)
    mean /= np.pi * radius ** 2
    var, _ = dblquad(lambda y, x: y * np.power(_perturbed(y, x) - mean, 2), 0, 2 * np.pi, 0, radius, epsabs=0.001 ** 2)
    var /= np.pi * radius ** 2
    stddev = np.sqrt(var)
    
    return mean, stddev

Test for the channel

In [4]:
%%time
center_stopping_power = traj_computer.compute_stopping_power([0,0.75,0.75], [1,0,0], 1.)[0]
center_stopping_power

CPU times: user 1.59 s, sys: 2.74 s, total: 4.33 s
Wall time: 3.07 s


0.23678958527234162

In [5]:
%%time
channel_mean, channel_std = compute_stopping_distribution([0,0.75,0.75], [1,0,0], 1., 0.03)
channel_mean, channel_std

CPU times: user 29min 3s, sys: 33min 42s, total: 1h 2min 45s
Wall time: 21min 13s


(0.23680364625218955, 5.689943179304135e-06)